In [1]:
import cv2
import numpy as np

img = cv2.imread('/home/hp/Documents/Daily_Task/Day_2/uv_nov11/5296_uv.png')
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

lower_blue = np.array([100, 50, 50])
upper_blue = np.array([130, 255, 255])

mask = cv2.inRange(hsv, lower_blue, upper_blue)

In [2]:
kernel = np.ones((6, 6), np.uint8)
mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

In [3]:
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

In [7]:
if contours:
    largest_contour = max(contours, key=cv2.contourArea)
    hull = cv2.convexHull(largest_contour)
    x, y, w, h = cv2.boundingRect(largest_contour)
    (cx, cy), radius = cv2.minEnclosingCircle(largest_contour)

    padding = 40
    x_bbox = max(0, x - padding)
    y_bbox = max(0, y - padding)
    w_bbox = w + 2 * padding
    h_bbox = h + 2 * padding

    w_bbox = min(w_bbox, img.shape[1] - x_bbox)
    h_bbox = min(h_bbox, img.shape[0] - y_bbox)
    
    yarn_mask = np.zeros_like(mask)
    cv2.drawContours(yarn_mask, [hull], -1, 255, -1)
    
    yarn_cone_mask = yarn_mask[y_bbox:y_bbox+h_bbox, x_bbox:x_bbox+w_bbox]
    yarn_cone_img = img[y_bbox:y_bbox+h_bbox, x_bbox:x_bbox+w_bbox]
    

    yarn_cone_isolated = cv2.bitwise_and(yarn_cone_img, yarn_cone_img, mask=yarn_cone_mask)
    
    cv2.rectangle(img, (x_bbox, y_bbox), (x_bbox+w_bbox, y_bbox+h_bbox), (0, 255, 0), 2)
    cv2.putText(img, 'Yarn Cone', (x_bbox, y_bbox-10), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

cv2.namedWindow('Original', cv2.WINDOW_NORMAL)
cv2.namedWindow('Yarn Cone Mask', cv2.WINDOW_NORMAL)
cv2.namedWindow('Isolated Yarn Cone', cv2.WINDOW_NORMAL)

cv2.imshow('Original', img)
cv2.imshow('Yarn Cone Mask', yarn_cone_mask)
cv2.imshow('Isolated Yarn Cone', yarn_cone_isolated)

cv2.imwrite('Yarn_mask.png', yarn_cone_mask)
cv2.imwrite('yarn.png', yarn_cone_isolated)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [1]:
import cv2
import numpy as np

img = cv2.imread('/home/hp/Documents/Daily_Task/Day_2/uv_nov11/5572_uv.png')
if img is None:
    print("Error: Image not found. Please check the file path.")
    exit()


img_display = img.copy() 


hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

lower_blue = np.array([100, 50, 50])
upper_blue = np.array([130, 255, 255])

mask_initial = cv2.inRange(hsv, lower_blue, upper_blue)

kernel = np.ones((6, 6), np.uint8)
mask_initial = cv2.morphologyEx(mask_initial, cv2.MORPH_CLOSE, kernel)
mask_initial = cv2.morphologyEx(mask_initial, cv2.MORPH_OPEN, kernel)

contours, _ = cv2.findContours(mask_initial, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

if contours:
    largest_contour = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(largest_contour)


    padding = 20
    x_bbox = max(0, x - padding)
    y_bbox = max(0, y - padding)
    w_bbox = w + 2 * padding
    h_bbox = h + 2 * padding
    
    w_bbox = min(w_bbox, img.shape[1] - x_bbox)
    h_bbox = min(h_bbox, img.shape[0] - y_bbox)
    

    rect = (x_bbox, y_bbox, w_bbox, h_bbox) # (x, y, w, h) format

    # Create a mask for GrabCut (must be same size as image)
    grabcut_mask = np.zeros(img.shape[:2], np.uint8) 
    
    # Create temporary background and foreground models
    bgdModel = np.zeros((1, 65), np.float64)
    fgdModel = np.zeros((1, 65), np.float64)

    cv2.grabCut(img,            
                grabcut_mask,  
                rect,           
                bgdModel,      
                fgdModel,      
                7,            
                cv2.GC_INIT_WITH_RECT)
    
    # === Final Mask ===
    # The grabcut_mask now has 4 values:
    # 0 (cv2.GC_BGD) = Obvious Background
    # 2 (cv2.GC_PR_BGD) = Probable Background
    # 1 (cv2.GC_FGD) = Obvious Foreground
    # 3 (cv2.GC_PR_FGD) = Probable Foreground
    

    final_mask = np.where((grabcut_mask == 2) | (grabcut_mask == 0), 0, 1).astype('uint8')
    kernel = np.ones((17, 17), np.uint8)
    final_mask_cleaned = cv2.morphologyEx(final_mask, cv2.MORPH_OPEN, kernel)

    yarn_cone_isolated = cv2.bitwise_and(img, img, mask=final_mask_cleaned)
    
    cv2.rectangle(img_display, (x_bbox, y_bbox), (x_bbox+w_bbox, y_bbox+h_bbox), (0, 255, 0), 2)
    cv2.putText(img_display, 'Yarn Cone (GrabCut)', (x_bbox, y_bbox-10), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    cv2.namedWindow('Original with BBox', cv2.WINDOW_NORMAL)
    cv2.namedWindow('Initial (Rough) Mask', cv2.WINDOW_NORMAL)
    cv2.namedWindow('GrabCut Final Mask', cv2.WINDOW_NORMAL)
    cv2.namedWindow('Isolated Yarn Cone', cv2.WINDOW_NORMAL)

    cv2.imshow('Original with BBox', img_display)
    cv2.imshow('Initial (Rough) Mask', mask_initial)
    cv2.imshow('GrabCut Final Mask', final_mask_cleaned * 255) 
    cv2.imshow('Isolated Yarn Cone', yarn_cone_isolated)

    cv2.imwrite('Yarn_mask_grabcut.png', final_mask * 255)
    cv2.imwrite('yarn_grabcut.png', yarn_cone_isolated)

    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
else:
    print("No contours found. Check your color thresholding values or image path.")

In [25]:
import cv2
import numpy as np

image_path = '/home/hp/Documents/Daily_Task/Day_2/uv_nov11/5356_uv.png'
img = cv2.imread(image_path)
if img is None:
    print(f"Error: Image not found at path: {image_path}")
    exit()

img_display = img.copy() 
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

lower_blue = np.array([100, 50, 50])
upper_blue = np.array([130, 255, 255])

mask_initial = cv2.inRange(hsv, lower_blue, upper_blue)

kernel_initial = np.ones((7, 7), np.uint8)
mask_initial = cv2.morphologyEx(mask_initial, cv2.MORPH_CLOSE, kernel_initial)
mask_initial = cv2.morphologyEx(mask_initial, cv2.MORPH_OPEN, kernel_initial)

contours, hierarchy = cv2.findContours(mask_initial, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

if contours:
    largest_contour_index = -1
    max_area = 0
    
    for i, (contour, h) in enumerate(zip(contours, hierarchy[0])):
        if h[3] == -1: 
            area = cv2.contourArea(contour)
            if area > max_area:
                max_area = area
                largest_contour_index = i

    if largest_contour_index == -1:
        print("No top-level contour found.")
        exit()
        
    largest_contour_initial = contours[largest_contour_index]
    (_, _), radius = cv2.minEnclosingCircle(largest_contour_initial)
    x, y, w, h = cv2.boundingRect(largest_contour_initial)
    
    inner_contour_index = hierarchy[0][largest_contour_index][2]
    
    (cx_outer, cy_outer), _ = cv2.minEnclosingCircle(largest_contour_initial)
    center_coords = (int(cx_outer), int(cy_outer))
    
    if inner_contour_index != -1:
        inner_contour = contours[inner_contour_index]
        M = cv2.moments(inner_contour)
        if M["m00"] > 0:
                        cx_inner = int(M["m10"] / M["m00"])
                        cy_inner = int(M["m01"] / M["m00"])
                        distance = np.hypot(cx_inner - cx_outer, cy_inner - cy_outer)
                        max_allowed_distance = radius * 0.25 
                        
                        if distance < max_allowed_distance:
                            center_coords = (cx_inner, cy_inner) 
                            print(f"Success: Inner centroid PASSED check (Dist: {distance:.1f}). Using as center: {center_coords}")
                            cv2.circle(img_display, center_coords, 5, (255, 0, 0), -1) 
                        else:
                            print(f"Warning: Inner centroid FAILED check (Dist: {distance:.1f} > Max: {max_allowed_distance:.1f}). Using outer center.")
                            cv2.circle(img_display, center_coords, 5, (0, 255, 255), -1)
    else:
        print("Warning: No inner hole detected. Using outer center.")
        
    padding = 20
    x_bbox = max(0, x - padding)
    y_bbox = max(0, y - padding)
    w_bbox = w + 2 * padding
    h_bbox = h + 2 * padding
    w_bbox = min(w_bbox, img.shape[1] - x_bbox)
    h_bbox = min(h_bbox, img.shape[0] - y_bbox)
    
    perfect_circle_mask = np.zeros(img.shape[:2], np.uint8)
    radius_int = int(radius) - 10 
    

    cv2.circle(perfect_circle_mask, center_coords, radius_int, 255, -1)
    

    yarn_cone_isolated = cv2.bitwise_and(img, img, mask=perfect_circle_mask)
    
    cv2.rectangle(img_display, (x_bbox, y_bbox), (x_bbox+w_bbox, y_bbox+h_bbox), (0, 255, 0), 2)
    cv2.putText(img_display, 'Yarn Cone', (x_bbox, y_bbox-10), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    cv2.namedWindow('Yarn Cone', cv2.WINDOW_NORMAL)
    cv2.imshow('Yarn Cone', yarn_cone_isolated)
    cv2.imwrite('yarn.png', yarn_cone_isolated)

    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
else:
    print("No initial contours found. Check your color thresholding values or image path.")

Success: Inner centroid PASSED check (Dist: 20.7). Using as center: (1041, 492)
